<a href="https://colab.research.google.com/github/DaveManning/d3js-essential-training-6007283/blob/main/Scenario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Create an interactive dashboard to visualize financial performance metrics (Gross Revenue, Net Margins, Net Revenue Retention, and CAC Payback) over 'Quarters' from the `/content/QuarterlyPnL_data.csv` file, with a dropdown selector to switch between different scenario types.

## Load and Prepare Data

### Subtask:
Load the `/content/QuarterlyPnL_data.csv` file into a pandas DataFrame. Inspect the data's structure, column names, and data types, focusing on 'Quarters', the scenario types, and the four financial metrics (Gross Revenue, Net Margins, Net Revenue Retention, CAC Payback). Perform any necessary data cleaning or transformations, such as converting 'Quarters' to a suitable time series format and ensuring scenario types are correctly categorized, to prepare the data for interactive plotting.


**Reasoning**:
First, I'll import the pandas library to handle data manipulation and load the `/content/QuarterlyPnL_data.csv` file into a DataFrame. Then I'll display the first few rows to quickly assess the data's structure and content, and use `.info()` to get a summary of the DataFrame, including column names, non-null counts, and data types, which is crucial for identifying initial cleaning and conversion needs.



In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/QuarterlyPnL_data.csv')

# Display the first few rows of the DataFrame
print("First 5 rows of the DataFrame:")
print(df.head())

# Display column names and their data types
print("\nDataFrame Info:")
df.info()

First 5 rows of the DataFrame:
                                                Date  Quarter     Scenario  \
0  Sat Mar 30 2024 20:00:00 GMT-0400 (Eastern Day...  2024-Q1         Base   
1  Sat Mar 30 2024 20:00:00 GMT-0400 (Eastern Day...  2024-Q1   Optimistic   
2  Sat Mar 30 2024 20:00:00 GMT-0400 (Eastern Day...  2024-Q1  Pessimistic   
3  Sat Jun 29 2024 20:00:00 GMT-0400 (Eastern Day...  2024-Q2         Base   
4  Sat Jun 29 2024 20:00:00 GMT-0400 (Eastern Day...  2024-Q2   Optimistic   

   Gross Revenue  Net Margins  Net Revenue Retention  CAC Payback  
0        2000000       0.2067                 1.0949         11.2  
1        2000000       0.2389                 1.3450          8.7  
2        2000000       0.1196                 0.9107         17.8  
3        2070200       0.2136                 1.1155         11.4  
4        2160400       0.2473                 1.3256          8.9  

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data 

**Reasoning**:
Based on the previous output, the 'Quarter' column is currently an object type and needs to be converted into a datetime format for time series analysis. Also, I need to identify the unique values in the 'Scenario' column to understand the different categories for the dashboard's dropdown selector. Finally, I will verify that the financial metrics are numeric, although the previous `df.info()` already showed them to be float64 or int64.



In [ ]:
import pandas as pd

# Convert 'Quarter' to datetime objects. The format '%Y-Q%q' can be used with pandas PeriodIndex
# However, for plotting, converting to the start of the quarter as datetime is more common and flexible.
# We can convert 'YYYY-QZ' to a datetime by treating it as the first day of that quarter.
df['Quarter'] = pd.to_datetime(df['Quarter'].str.replace('Q1', '-01-01').str.replace('Q2', '-04-01').str.replace('Q3', '-07-01').str.replace('Q4', '-10-01'))

# Identify unique scenario types
unique_scenarios = df['Scenario'].unique()

# Verify data types of financial metrics (already checked by df.info() but good to re-confirm specific columns)
financial_metrics = ['Gross Revenue', 'Net Margins', 'Net Revenue Retention', 'CAC Payback']
metric_dtypes = df[financial_metrics].dtypes

print("\n'Quarter' column after conversion (first 5 rows):")
print(df['Quarter'].head())
print("\nData type of 'Quarter' column:", df['Quarter'].dtype)
print("\nUnique Scenario types:")
print(unique_scenarios)
print("\nData types of Financial Metrics:")
print(metric_dtypes)


'Quarter' column after conversion (first 5 rows):
0   2024-01-01
1   2024-01-01
2   2024-01-01
3   2024-04-01
4   2024-04-01
Name: Quarter, dtype: datetime64[ns]

Data type of 'Quarter' column: datetime64[ns]

Unique Scenario types:
['Base' 'Optimistic' 'Pessimistic']

Data types of Financial Metrics:
Gross Revenue              int64
Net Margins              float64
Net Revenue Retention    float64
CAC Payback              float64
dtype: object


## Create Interactive Financial Dashboard

### Subtask:
Design an interactive dashboard using a suitable plotting library like Plotly or Altair. This dashboard will feature four individual line charts, each representing one of the financial metrics: Gross Revenue, Net Margins, Net Revenue Retention, and CAC Payback, plotted against 'Quarters' on the x-axis. Implement a single dropdown selector using `ipywidgets` or the plotting library's selection features. This selector will allow the user to choose between the different scenario types available in the data, with all four charts dynamically updating to display the data for the selected scenario. Ensure each chart has clear titles, labeled axes, and appropriate legends.


**Reasoning**:
I will import the necessary libraries, altair for plotting and ipywidgets for the interactive dropdown, as instructed. This prepares the environment for creating the interactive dashboard.



In [ ]:
import altair as alt
import ipywidgets as widgets
from IPython.display import display

print("Libraries imported: altair, ipywidgets, IPython.display")

Libraries imported: altair, ipywidgets, IPython.display


**Reasoning**:
I will create an interactive dashboard using Altair and ipywidgets, following the instructions to create a dropdown selector for scenarios and four dynamically updating line charts for the financial metrics. The `update_dashboard` function will filter the data based on the selected scenario and generate the Altair charts, which will then be displayed using `ipywidgets.interactive`.



In [ ]:
financial_metrics = ['Gross Revenue', 'Net Margins', 'Net Revenue Retention', 'CAC Payback']

# Create the dropdown widget
scenario_selector = widgets.Dropdown(
    options=unique_scenarios.tolist(), # Convert numpy array to list for dropdown options
    value='Base', # Default selected value
    description='Scenario:',
    disabled=False,
)

def update_dashboard(selected_scenario):
    # Filter the DataFrame based on the selected scenario
    filtered_df = df[df['Scenario'] == selected_scenario]

    charts = []
    for metric in financial_metrics:
        # Create a line chart for each financial metric
        chart = alt.Chart(filtered_df).mark_line(point=True).encode(
            x=alt.X('Quarter:T', axis=alt.Axis(title='Quarter')),
            y=alt.Y(metric, axis=alt.Axis(title=metric)),
            tooltip=['Quarter:T', alt.Tooltip(metric, format=',' if metric == 'Gross Revenue' else '.2f')]
        ).properties(
            title=f'{metric} Over Quarters ({selected_scenario} Scenario)'
        ).interactive()
        charts.append(chart)

    # Combine the charts into a single dashboard
    # Using vconcat to stack them vertically for better readability
    combined_chart = alt.vconcat(*charts)
    display(combined_chart)

# Display the dropdown and link it to the update_dashboard function
# Use widgets.interactive to create the interactive output
interactive_dashboard = widgets.interactive(update_dashboard, selected_scenario=scenario_selector)
display(interactive_dashboard)


interactive(children=(Dropdown(description='Scenario:', options=('Base', 'Optimistic', 'Pessimistic'), value='…

## How to Use the Interactive Dashboard

This dashboard allows you to analyze key financial performance metrics across different business scenarios: 'Base', 'Optimistic', and 'Pessimistic'.

### Using the Scenario Selector

1.  **Locate the Dropdown Menu**: At the top of the dashboard, you will find a dropdown menu labeled "Scenario:".
2.  **Select a Scenario**: Click on the dropdown menu and choose one of the three available scenarios: 'Base', 'Optimistic', or 'Pessimistic'.
3.  **Dynamic Updates**: Once you select a scenario, all four charts below (Gross Revenue, Net Margins, Net Revenue Retention, and CAC Payback) will automatically update to display the data corresponding to the chosen scenario.

### Interpreting the Visualizations

*   **Gross Revenue**: This chart shows the total revenue generated over quarters. Observe how revenue trends differ between scenarios (e.g., higher growth in 'Optimistic' vs. slower growth or decline in 'Pessimistic').
*   **Net Margins**: This metric indicates the profitability. Higher net margins are generally better. Compare how different scenarios impact the company's ability to retain profit from revenue.
*   **Net Revenue Retention**: This shows the percentage of recurring revenue retained from existing customers. A value above 1.0 (or 100%) indicates growth from existing customers. Analyze how retention rates vary across scenarios.
*   **CAC Payback**: This measures how many months it takes to recoup the cost of acquiring a customer. Lower numbers are better. The charts will show how customer acquisition efficiency changes under different market conditions represented by the scenarios.

By switching between scenarios, you can quickly identify potential risks and opportunities, understand the sensitivity of your financial performance to different assumptions, and make more informed strategic decisions.

## Final Task

### Subtask:
Provide a summary of the created interactive dashboard, explaining how to effectively use the scenario selector to analyze the financial performance metrics under different scenarios. This will include guidance on interpreting the trends and insights revealed by the interactive visualizations.


## Summary:

### Data Analysis Key Findings

*   The dataset `/content/QuarterlyPnL_data.csv` was successfully loaded, containing quarterly financial data across different scenarios. Key columns include `Quarter`, `Scenario`, `Gross Revenue`, `Net Margins`, `Net Revenue Retention`, and `CAC Payback`.
*   The `Quarter` column was successfully transformed from an `object` type (e.g., '2024-Q1') to a `datetime64[ns]` format, setting the date to the first day of each respective quarter, which is suitable for time-series analysis.
*   The `Scenario` column was identified to contain three unique categories: 'Base', 'Optimistic', and 'Pessimistic', which serve as the options for the interactive scenario selector.
*   An interactive dashboard was successfully implemented using Altair and ipywidgets. This dashboard features a dropdown selector that allows users to switch between the 'Base', 'Optimistic', and 'Pessimistic' scenarios.
*   The dashboard dynamically displays four line charts, visualizing the trends of 'Gross Revenue', 'Net Margins', 'Net Revenue Retention', and 'CAC Payback' over quarters, updating all charts simultaneously based on the selected scenario.

### Insights or Next Steps

*   The interactive dashboard provides a dynamic tool for stakeholders to quickly analyze and compare financial performance metrics under various strategic scenarios, aiding in robust financial planning and risk assessment.
*   The next step involves creating a comprehensive summary of the dashboard's functionality, detailing how to effectively use the scenario selector to interpret financial trends and derive insights from the visualizations.
